In [1]:
!pip install -q youtube-comment-downloader yt-dlp pandas tqdm

In [2]:
# ── 설정 ──────────────────────────────────────────────────
TARGET_SIZE = 100_000  # target number of videos to collect
BATCH_SIZE = 1000  # number of videos to fetch per query
MAX_DURATION = 60  # max duration in seconds (for shorts)

OUTPUT_DIR = "youtube_shorts_dataset"
ID_OUTPUT_PATH = f"{OUTPUT_DIR}/video_ids.csv"
# ──────────────────────────────────────────────────────────

In [3]:
# ── Step 0. 기존 ID 로드 (중복 방지) ─────────────────────
import pandas as pd
import os

os.makedirs(OUTPUT_DIR, exist_ok=True)

if os.path.exists(ID_OUTPUT_PATH):
    existing_df = pd.read_csv(ID_OUTPUT_PATH)
    seen = set(existing_df["video_id"].dropna().tolist())
    print(
        f"Loaded existing IDs: {len(seen)} -> skipping already collected videos"
    )
else:
    seen = set()
    print("No existing file found -> starting fresh")

print(
    f"Target: {TARGET_SIZE} / Current: {len(seen)} / Need to add: {max(0, TARGET_SIZE - len(seen))}"
)

Loaded existing IDs: 3432 -> skipping already collected videos
Target: 100000 / Current: 3432 / Need to add: 96568


In [ ]:
# ── Step 1. Define search query list ─────────────────────
# Query diversity directly improves dataset diversity

GENERAL_QUERIES = [
    # viral
    "funny",
    "viral",
    "trending",
    "satisfying",
    # meme
    "meme",
    "funny meme",
    "relatable",
    # food
    "food",
    "cooking",
    "recipe",
    "asmr food",
    # animals
    "animals",
    "cute dog",
    "dog",
    "cute cat",
    "cat",
    # sports
    "sports",
    "basketball",
    "soccer",
    "gym",
    # games
    "gaming",
    "minecraft",
    "roblox",
    # beauty/fashion
    "makeup",
    "fashion",
    "outfit",
    # music
    "music",
    "dance",
    "singing",
    # education
    "did you know",
    "facts",
    "life hack",
    # etc
    "prank",
    "challenge",
    "travel",
    "nature",
    "science",
    "art",
    "diy",
    "comedy",
]

TRENDING_QUERIES = [
    # --------- treding.google.com filtering youtube
    # trending topics (2026)
    "half-life",
    "ai",
    "olympics",
    "bad bunny",
    # trending topics (2025)
    "kpop demon hunters",
    "soda pop",
    "67",
    # ---------
]

MADE_UP_QUERIES = [
    "fyp",
    "",  # for general shorts without specific keywords
]

# BASE_QUERIES = GENERAL_QUERIES + TRENDING_QUERIES + MADE_UP_QUERIES
BASE_QUERIES = MADE_UP_QUERIES

QUERIES = []
for q in BASE_QUERIES:
    QUERIES.append(f"{q} #shorts")

print(f"Total number of queries: {len(QUERIES)}")
print(
    f"{BATCH_SIZE} results per query → max candidates: {len(QUERIES) * BATCH_SIZE:,}"
)
print("Sample queries:", QUERIES[:5])

Total number of queries: 48
1000 results per query → max candidates: 48,000
Sample queries: ['funny #shorts', 'viral #shorts', 'trending #shorts', 'satisfying #shorts', 'meme #shorts']


In [5]:
# ── Step 2. Collect IDs by iterating through queries ─────────
import yt_dlp

ydl_opts = {
    "quiet": True,
    "extract_flat": True,
    "skip_download": True,
}

newly_collected = []
need = TARGET_SIZE - len(seen)

for query in QUERIES:
    if len(newly_collected) >= need:
        break

    print(
        f"[{len(seen) + len(newly_collected)}/{TARGET_SIZE}] Searching: '{query}'"
    )

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            result = ydl.extract_info(
                f"ytsearch{BATCH_SIZE}:{query}", download=False
            )
        entries = result.get("entries", [])
    except Exception as ex:
        print(f"  ↳ error: {ex}")
        continue

    for e in entries:
        if len(newly_collected) >= need:
            break

        vid = e.get("id")
        if not vid or vid in seen:
            continue

        duration = e.get("duration")
        if duration is not None and duration > MAX_DURATION:
            continue

        seen.add(vid)
        newly_collected.append(
            {
                "video_id": vid,
                "title": e.get("title", ""),
                "duration": duration,
                "view_count": e.get("view_count"),
                "url": f"https://www.youtube.com/shorts/{vid}",
                "query": query,
            }
        )

print(f"\nCompleted: {len(newly_collected)} new videos collected")

[3432/100000] Searching: 'funny #shorts'
[3586/100000] Searching: 'viral #shorts'
[3960/100000] Searching: 'trending #shorts'
[4258/100000] Searching: 'satisfying #shorts'
[4412/100000] Searching: 'meme #shorts'
[4720/100000] Searching: 'funny meme #shorts'
[4870/100000] Searching: 'relatable #shorts'
[5412/100000] Searching: 'food #shorts'
[5624/100000] Searching: 'cooking #shorts'
[5736/100000] Searching: 'recipe #shorts'
[5792/100000] Searching: 'asmr food #shorts'
[5912/100000] Searching: 'animals #shorts'
[5918/100000] Searching: 'cute dog #shorts'
[6102/100000] Searching: 'dog #shorts'
[6460/100000] Searching: 'cute cat #shorts'
[6590/100000] Searching: 'cat #shorts'
[6898/100000] Searching: 'sports #shorts'
[7158/100000] Searching: 'basketball #shorts'
[7636/100000] Searching: 'soccer #shorts'
[7826/100000] Searching: 'gym #shorts'
[8220/100000] Searching: 'gaming #shorts'
[8410/100000] Searching: 'minecraft #shorts'
[8522/100000] Searching: 'roblox #shorts'
[8620/100000] Search

[9490/100000] Searching: 'singing #shorts'
[9616/100000] Searching: 'did you know #shorts'


ERROR: Unable to download API page: HTTP Error 403: Forbidden (caused by <HTTPError 403: Forbidden>)


  ↳ error: ERROR: Unable to download API page: HTTP Error 403: Forbidden (caused by <HTTPError 403: Forbidden>)
[9616/100000] Searching: 'facts #shorts'


ERROR: Unable to download API page: HTTP Error 403: Forbidden (caused by <HTTPError 403: Forbidden>)


...

ERROR: Unable to download API page: HTTP Error 403: Forbidden (caused by <HTTPError 403: Forbidden>)


  ↳ error: ERROR: Unable to download API page: HTTP Error 403: Forbidden (caused by <HTTPError 403: Forbidden>)
[9778/100000] Searching: 'science #shorts'


ERROR: Unable to download API page: HTTP Error 403: Forbidden (caused by <HTTPError 403: Forbidden>)


  ↳ error: ERROR: Unable to download API page: HTTP Error 403: Forbidden (caused by <HTTPError 403: Forbidden>)
[9778/100000] Searching: 'art #shorts'


KeyboardInterrupt: 

In [ ]:
# ── Step 3. Append new data to existing CSV ───────────────────────
new_df = pd.DataFrame(newly_collected)

if os.path.exists(ID_OUTPUT_PATH) and len(newly_collected) > 0:
    new_df.to_csv(ID_OUTPUT_PATH, mode="a", header=False, index=False)
    print(f"Appended {len(new_df)} new videos → {ID_OUTPUT_PATH}")
elif len(newly_collected) > 0:
    new_df.to_csv(ID_OUTPUT_PATH, index=False)
    print(f"Saved {len(new_df)} new videos → {ID_OUTPUT_PATH}")
else:
    print("No new videos collected")

total = pd.read_csv(ID_OUTPUT_PATH)
print(f"Total accumulated: {len(total)} videos")
total.tail()

Appended 459 new videos → youtube_shorts_dataset/video_ids.csv
Total accumulated: 3432 videos


,video_id,title,duration,view_count,url,query
3427,op2UEKnb5lA,#shorts #scary #67,19.0,27.0,https://www.youtube.com/shorts/op2UEKnb5lA,67 #shorts
3428,8LJCHif3vLE,Shot On iPhone Meme 67 ||pee poo #shorts #tre...,38.0,221.0,https://www.youtube.com/shorts/8LJCHif3vLE,67 #shorts
3429,k1eiUpe-HvM,67 gassy #meme #shorts,19.0,19.0,https://www.youtube.com/shorts/k1eiUpe-HvM,67 #shorts
3430,0QolB-ubGHQ,Big bank TikTok challenge #67 #shorts,6.0,1.0,https://www.youtube.com/shorts/0QolB-ubGHQ,67 #shorts
3431,dH81vXDB1Pg,big bank challenge 🍩😋 tiktok #shorts #tiktok b...,26.0,4.0,https://www.youtube.com/shorts/dH81vXDB1Pg,67 #shorts
